In [4]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


training = pd.read_csv('Training.csv')
testing  = pd.read_csv('Testing.csv')
cols     = training.columns
cols     = cols[:-1]
x        = training[cols]
y        = training['prognosis']
y1       = y

reduced_data = training.groupby(training['prognosis']).max()

#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']  
testy    = le.transform(testy)

In [6]:
clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
print(clf.score(x_train,y_train))
print ("cross result========")
scores = model_selection.cross_val_score(clf, x_test, y_test, cv=3)
print (scores)
print (scores.mean())
print(clf.score(testx,testy))

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

#feature_importances
for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1.0
cross result========
[0.9704797  0.97966728 0.97042514]
0.9735240420796075
1.0
1. feature 54 - runny_nose (0.026703)
2. feature 93 - internal_itching (0.026649)
3. feature 79 - hip_joint_pain (0.026626)
4. feature 105 - polyuria (0.026390)
5. feature 128 - inflammatory_nails (0.025712)
6. feature 71 - enlarged_thyroid (0.025479)
7. feature 96 - irritability (0.024950)
8. feature 118 - blood_in_sputum (0.024874)
9. feature 131 - yellow_crust_ooze (0.024798)
10. feature 2 - nodal_skin_eruptions (0.024735)


In [7]:
print("Please reply Yes or No for the following symptoms") 
def print_disease(node):
    #print(node)
    node = node[0]
    #print(len(node))
    val  = node.nonzero() 
    #print(val)
    disease = le.inverse_transform(val[0])
    return disease
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    #print(tree_)
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    #print("def tree({}):".format(", ".join(feature_names)))
    symptoms_present = []
    def recurse(node, depth):
        
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(name + " ?")
            ans = input()
            ans = ans.lower()
            if ans == 'yes':
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                #symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            for i in symptoms_given:
                print(i +"?")
                af= input()
                af=af.lower()
                if af=='yes':
                    symptoms_present.append(i)
            print( "You may have " +  present_disease )
            print("symptoms given "  +  str(list(symptoms_given)) )
            print("symptoms present  " + str(list(symptoms_present)))
            confidence_level = (1.0*len(symptoms_present)/len(symptoms_given))
            print("confidence level is " + str(confidence_level))
    recurse(0, 1)
tree_to_code(clf,cols)

Please reply Yes or No for the following symptoms
runny_nose ?
Yes
continuous_sneezing?
No
chills?
Yes
fatigue?
no
cough?
yes
high_fever?
no
headache?
yes
swelled_lymph_nodes?
not
malaise?
yes
phlegm?
no
throat_irritation?
yes
redness_of_eyes?
no
sinus_pressure?
yes
runny_nose?
no
congestion?
yes
chest_pain?
no
loss_of_smell?
yes
muscle_pain?
yes
['You may have Common Cold']
symptoms given ['continuous_sneezing', 'chills', 'fatigue', 'cough', 'high_fever', 'headache', 'swelled_lymph_nodes', 'malaise', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'loss_of_smell', 'muscle_pain']
symptoms present  ['chills', 'cough', 'headache', 'malaise', 'throat_irritation', 'sinus_pressure', 'congestion', 'loss_of_smell', 'muscle_pain']
confidence level is 0.5294117647058824


# 